In [ ]:
"""
Making figures for fraction SiOH plots
"""
import numpy as np

import complete_edl as edl
import constants as C

%matplotlib widget

DEFAULT_GAMMA = 4
DEFAULT_CONC = 1e-1  # 100 mM
GAMMA_RANGE = [3, 4, 5, 6]
CONC_RANGE = [10e-3, 100e-3, 1000e-3]
PH_RANGE = np.linspace(2, 13, 200)


def calculate_f_sioh(ph_sweep_sol):
    """
    Calculate f_SiOH
    """
    frac_sioh = (C.N_SITES_SILICA + ph_sweep_sol["charge"] / C.E_0) / C.N_SITES_SILICA
    return frac_sioh


ph_range = np.linspace(2, 13, 200)
frac_sioh_gamm = np.zeros((len(GAMMA_RANGE), len(PH_RANGE)))

for i, gamma in enumerate(GAMMA_RANGE):
    silica = edl.Aqueous(DEFAULT_CONC, gamma, DEFAULT_GAMMA, DEFAULT_GAMMA, 3)
    silica_sol = silica.ph_sweep(ph_range, tol=1e-2)
    frac_sioh_gamm[i, :] = calculate_f_sioh(silica_sol)

frac_sioh_conc = np.zeros((len(CONC_RANGE), len(ph_range)))
for i, conc in enumerate(CONC_RANGE):
    silica = edl.Aqueous(conc, DEFAULT_GAMMA, DEFAULT_GAMMA, DEFAULT_GAMMA, 3)
    silica_sol = silica.ph_sweep(ph_range, tol=1e-2)
    frac_sioh_conc[i, :] = (
        C.N_SITES_SILICA + silica_sol["charge"] / C.E_0
    ) / C.N_SITES_SILICA



In [ ]:
import pandas as pd

gamma_df = pd.DataFrame(
    data=frac_sioh_gamm.T, 
    columns=[f'gamma = {gamma}' for gamma in GAMMA_RANGE], 
    index=ph_range
)

conc_df = pd.DataFrame(
    data=frac_sioh_conc.T, 
    columns=[f'conc = {conc*1e3:.0f} mM' for conc in CONC_RANGE], 
    index=ph_range
)

all_df = gamma_df.join(conc_df, how='left')

all_df.index.name = 'pH'

all_df.to_csv('f_sioh_data.csv')